In [ ]:
import sys
import numpy as np
import scipy
import subprocess
from natsort import natsorted
import itertools
import pandas as pd
import torch
sys.path.append("../utls")
sys.path.append("../utls")
sys.path.append("../.")
sys.path.append("../models")
from collections import Counter
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torchvision
import torchvision.transforms
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
from utls.preprocessing import *
from utls.postprocessing import get_dataset, generate_hist_df
from InterfaceDeclaration import LPBFInterface
from models.MLUtls import fade_in_out, standardize_tensor, getKFoldCrossValidationIndexes, train_log, transform_ft, dataset_by_cross_validation, labels_by_classes, get_current_fold_and_hist, LPBFDataset
from models.MLModels import SVMModel, CNN_Base_1D_Model, ResNet15_1D_Model

alphabet = list(string.ascii_lowercase)

project_name = ["MuSIC", "MaPS", "MuSIC_EXP1"]
if os.name == "posix":
    data_dir = subprocess.getoutput("echo $DATADIR")
elif os.name == "nt":
    data_dir = subprocess.getoutput("echo %datadir%")
music_dir = os.path.join(data_dir, "MuSIC")
if not os.path.exists(music_dir):
    project_name[0] = "2024-MUSIC"
daq_dir = os.path.join(data_dir, *project_name, "Acoustic Monitoring")
lmq_dir = os.path.join(data_dir, *project_name, "LMQ Monitoring")
del music_dir

sampling_rate_daq: int = int(1.25 * 1e6)
sampling_rate_lmq: int = int(0.1 * 1e6)
tdms_daq_list = natsorted(
    [i for i in os.listdir(daq_dir) if i.split(".")[-1] == "tdms"]
)
bin_lmq_list = natsorted([i for i in os.listdir(lmq_dir) if i.split(".")[-1] == "bin"])
lmq_channel_name = [
    "Vector ID",
    "meltpooldiode",
    "X Coordinate",
    "Y Coordinate",
    "Laser power",
    "Spare",
    "Laser diode",
    "Varioscan(focal length)",
]
process_regime = [
    [0,    59, "Base"  ], 
    [60,  129, "KH"    ], 
    [130, 199, "Normal"], 
    [200, 269, "RLoF"  ], 
    [269, 339, "LoF"   ] 
]

dataset,sc_power,le_direction,le_speed,le_region = get_dataset()

with open(os.path.join("../",'outputs',"intermediate",f"sc_power.pkl"), 'rb') as handle:
    sc_power = pickle.load(handle)

with open(os.path.join("../",'outputs',"intermediate",f"le_speed.pkl"), 'rb') as handle:
    le_speed = pickle.load(handle)

with open(os.path.join("../",'outputs',"intermediate",f"le_region.pkl"), 'rb') as handle:
    le_region = pickle.load(handle)

with open(os.path.join("../",'outputs',"intermediate",f"le_direction.pkl"), 'rb') as handle:
    le_direction = pickle.load(handle)

In [3]:
max_epochs=50
his_dir = os.path.join(os.pardir,"lfs","weights/hist")
snap_dir = os.path.join(os.pardir,"lfs","weights/")
model_name= ["CNN"]
acoustic_type =['ae','mic'] 
context_type = []
# context_type = ['energy']
output_type = ['direction']
folds = 10

df = generate_hist_df(his_dir,model_name,acoustic_type,context_type,output_type,folds,max_epochs)
df['Input Type'] = df['Input type'].str.replace('+', '\n')

In [5]:
data_loader = DataLoader(dataset, batch_size=1000, shuffle=False)

In [6]:
labels = []
for _cube_position, _laser_power, _scanning_speed, _regime_info, _print_direction, _mic, _ae, _defect_labels in data_loader:  # Iterate over the dataset
    targets = _print_direction
    labels.extend(targets.numpy())  # Collect all labels (assumes targets are a tensor)
# Count occurrences of each class
class_counts = Counter(labels)
total_samples = sum(class_counts.values())

# Display the distribution as percentages
class_distribution = {cls: count / total_samples * 100 for cls, count in class_counts.items()}
print("Class Distribution (in %):", class_distribution)

Class Distribution (in %): {4: 22.366357575416387, 0: 22.381637888743235, 1: 27.49973862621941, 3: 27.572119057767626, 2: 0.18014685185334117}


In [10]:
_ae.shape

torch.Size([343, 11777])

In [7]:
model,snap_list = read_trained_model(snap_dir,model_name,acoustic_type,context_type,output_type,folds,max_epochs)
model = model.double()

In [ ]:
_model_name= "CNN"
_inputs = "ae"
_outputs = "direction"
meta_list = []
pred_list = []
label_list = [] 
ae_cf_direction = get_confusion_matrix(model, 'CNN','ae', 'direction')
mic_cf_direction = get_confusion_matrix(model, 'CNN','mic','direction')
ae_cf_position = get_confusion_matrix(model, 'CNN','ae', 'position')
mic_cf_position = get_confusion_matrix(model, 'CNN','mic','position')

In [9]:
np.save(f"../outputs/intermediate/ae_cf_direction.npy",ae_cf_direction)
np.save(f"../outputs/intermediate/mic_cf_direction.npy",mic_cf_direction)
np.save(f"../outputs/intermediate/ae_cf_position.npy",ae_cf_position)
np.save(f"../outputs/intermediate/mic_cf_position.npy",mic_cf_position)